## PN junction in one dimension
### Craig Lage 12Sep23

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

### 1D arrays to hold the data

In [ ]:
class Array1d:
    def __init__(self,xmin,xmax,nx):
        self.nx=nx
        self.xmin=xmin
        self.xmax=xmax
        self.dx=(xmax-xmin)/nx
        self.x=np.linspace(xmin+self.dx/2,xmax-self.dx/2,nx)
        self.data=np.zeros([nx])

### First set up and initialize the arrays.  Make the junction 10:1 asymmetric
### Linear dimensions are in microns, and doping densities in micron^-3
### Set initial values for the mobile charge densities and potentials.

In [ ]:
ni = 1.5E-2 # Silicon free carrier density in carriers/micron^3
qe = 1.6E-19 # Coulombs
kTq = 0.026 # kT/qe in eV at room T
eps = 11.7 * 8.85E-18 # Silicon dielectric constant in Farads/micron

xmin = -4.0
xmax = 4.0
nx = 1000

doping = Array1d(xmin, xmax, nx)
n = Array1d(xmin, xmax, nx)
p = Array1d(xmin, xmax, nx)
phi = Array1d(xmin, xmax, nx)
phiF = 0 # Fermi level

hsquared = doping.dx**2
Na = 1.0E3  # 1.0E15 cm^-3 on the P side
Nd = 1.0E2  # 1.0E14 cm^-3 on the N side
phiP = kTq * np.log(Na / ni)
phiN =  - kTq * np.log(Nd / ni)

for i in range(nx):
    if doping.x[i] < 0.0:
        doping.data[i] = -Na
        phi.data[i] = phiP
        p.data[i] = Na
        n.data[i] = 0.0
    else:
        doping.data[i] = Nd
        phi.data[i] = phiN
        p.data[i] = 0.0
        n.data[i] = Nd


In [ ]:
w = 1.8 # Successive over-relaxation factor - speeds convergence.
for j in range(10000):
    iterLimit = 100
    for i in range(2, nx-2):
        term1 = (phi.data[i+1] + phi.data[i-1] - hsquared * qe / eps * doping.data[i]) / 2.0
        mult = hsquared * ni * qe / (2.0 * eps)
        newphi = phi.data[i] #term1 - mult * np.exp(exponent) + mult * np.exp(-exponent)

        # Newton's method "Inner Loop"
        counter = 0
        tol = 1.0
        while ((tol > 1.0E-9) and (counter < iterLimit)):
            oldnewphi = newphi
            exponent = (newphi - phiF) / kTq
            f = newphi - term1 - mult * np.exp(-exponent) + mult * np.exp(exponent)
            fPrime = 1.0 + mult / kTq * np.exp(-exponent) + mult / kTq * np.exp(exponent)
            newphi = newphi - f / fPrime
            tol = abs(newphi - oldnewphi)
            counter += 1
        # Successive over-relaxation
        phi.data[i] = phi.data[i] + w * (newphi - phi.data[i])
        exponent = (phi.data[i] - phiF) / kTq
        n.data[i] = ni * np.exp(-exponent)
        p.data[i] = ni * np.exp(exponent)


In [ ]:
fig, axs = plt.subplots(2,1,figsize=(10,10))
axs[0].set_title("Charges")
axs[0].set_xlabel("X (microns)")
axs[0].set_ylabel("Carrier concentrations (microns^-3)")

axs[0].plot(doping.x, doping.data, ls='--', lw=4, label="Doping")
axs[0].plot(doping.x, n.data, label='Electrons')
axs[0].plot(doping.x, p.data, label='Holes')

# Plot theoretical depletion region widths
wP = np.sqrt(2.0 * eps / qe * Nd / Na / (Na + Nd) * (phiP - phiN))
wN = np.sqrt(2.0 * eps / qe * Na / Nd / (Na + Nd) * (phiP - phiN))
axs[0].plot([-wP,-wP],[0,1000], ls='--', color='black', label="P depletion width")
axs[0].plot([wN,wN],[0,1000], ls='--', color='black', label="N depletion width")
axs[0].legend()

axs[1].set_title("Potentials")
axs[1].set_xlabel("X (microns)")
axs[1].set_ylabel("Potential (V)")
axs[1].plot(doping.x, phi.data, label="Potential")
axs[1].plot([doping.xmin, doping.xmax], [0,0], ls='--', label="Fermi level")
axs[1].legend()